In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from datetime import datetime, timedelta  # 현재 날짜를 가져오기 위해 추가
import mysql.connector  # MySQL 연결을 위해 추가
import time

# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='13.211.174.98',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hanagift',   # MySQL 사용자 비밀번호
    database='db'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()


data_list = [] 
# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 4. "기간환율변동" 선택 클릭
try:
    period_exchange_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='inqType_p'].radioForm")))
    period_exchange_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='inqType_p']\").click();")


# 5. 날짜 수정 (조회 기간 시작일)
seven_days_before = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt_p")))
date_input.clear()
date_input.send_keys(seven_days_before)

today = datetime.now().strftime('%Y-%m-%d')
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqEndDt_p")))
date_input.clear()
date_input.send_keys(today)

# 6. 통화 선택 (코드 테이블에 있는 외화 목록을 반복 조회)
currency_codes = ['AUD', 'BRL', 'CAD', 'CNY', 'EUR', 'GBP', 'JPY', 'KRW', 'MYR', 'RUB', 'SEK', 'USD', 'VND']  # 테이블에서 확인한 코드들

currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
select = Select(currency_select)
select.select_by_value("USD")

# 7. "조회" 버튼 클릭
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
query_button.click()

# 8. 조회 후 5초 대기
time.sleep(90)



# 9. 테이블 데이터 크롤링
rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")

for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    data = [column.text for column in columns]
    
    # 첫 번째 값 제외, 나머지를 float로 변환
    for i in range(1, len(data)):
        try:
            data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
        except ValueError:
            print(f"변환 실패: {data[i]}")
    data_list.append(data)
    print(data)

# 웹 드라이버 종료
driver.quit()

['2024-09-11', 1367.82, 1320.78, 1357.4, 1331.2, 1328.61, 1344.3, 4.8, 6.96077, 1.0]
['2024-09-10', 1362.94, 1316.06, 1352.6, 1326.4, 1323.82, 1339.5, 9.1, 6.95393, 1.0]
['2024-09-09', 1353.68, 1307.12, 1343.4, 1317.4, 1314.83, 1330.4, 5.4, 6.95984, 1.0]
['2024-09-06', 1359.17, 1312.43, 1348.8, 1322.8, 1320.22, 1335.8, 6.2, 6.968, 1.0]
['2024-09-05', 1365.48, 1318.52, 1355.1, 1328.9, 1326.29, 1342.0, 1.0, 7.01249, 1.0]
['2024-09-04', 1364.46, 1317.54, 1354.1, 1327.9, 1325.29, 1341.0, 0.0, 7.02395, 1.0]


In [25]:
# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='13.211.174.98',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hanagift',   # MySQL 사용자 비밀번호
    database='db'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()

sql_query = """
INSERT INTO exchangeRate (exchangeDate, buyCash, sellCash, sendRemit, receiveRemit, buyForeignCheck, exchangeRate, previousComparison, rateGap, usdConversionRate)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 데이터 삽입
try:
    cursor.execute(sql_query, data)
    conn.commit()  # 변경사항을 데이터베이스에 반영
    print("데이터가 성공적으로 삽입되었습니다.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()  # 오류 발생 시 롤백

# 연결과 커서 닫기
cursor.close()
conn.close()

데이터가 성공적으로 삽입되었습니다.


In [27]:
# 오늘 날짜 가져오기
from datetime import datetime 
today = datetime.now().strftime('%Y-%m-%d')

In [28]:
today

'2024-09-11'